# Tutorial for flexx.event - properties and events

In [1]:
from flexx import event

### Events
In `flexx.event`, events are represented with dictionary objects that
provide information about the event (such as what button was pressed,
or the new value of a property). A custom `Dict` class is used that inherits from ``dict`` but allows attribute access,
e.g. ``ev.button`` as an alternative to ``ev['button']``.

### Handlers

Events originate from `HasEvents` objects. When an event is emitted, it is reacted upon by handlers. Handlers can be created in a few different ways, but here is the recommended approach:

In [2]:
class MyObject(event.HasEvents):
        
    @event.connect('foo')
    def on_foo(self, *events):
        print('received the foo event %i times' % len(events))

ob = MyObject()

In [3]:
ob.emit('foo', {})

Dict([('type', 'foo'), ('source', <__main__.MyObject at 0x1e09a21fb00>)])

In [4]:
ob.emit('foo', {}); ob.emit('foo', {})

received the foo event 1 times


Dict([('type', 'foo'), ('source', <__main__.MyObject at 0x1e09a21fb00>)])

Note how the handler connects using a "connection string", which (in this case) indicates we connect to the type "foo" event of the object. The connection string allows some powerful mechanics, as we will see later in this tutorial.

Also note how the handler accepts multiple events at once. This means that in situations where we only care about something being changed, we can skip "duplicate" events. In situation where each individual event needs processing, use `for ev in events: ...`.

Handlers also behave as functions. It may not always be applicable, but you can call a handler without any events:

In [5]:
ob.on_foo()

received the foo event 2 times
received the foo event 0 times


A handler can also connect to multiple events:

In [6]:
class MyObject(event.HasEvents):
        
    @event.connect('foo', 'bar')
    def on_foo_or_bar(self, *events):
        for ev in events:
            print('received the %s event' % ev.type)

ob = MyObject()
ob.emit('foo', {}); ob.emit('foo', {}); ob.emit('bar', {})

Dict([('type', 'bar'), ('source', <__main__.MyObject at 0x1e09a23e5c0>)])

Handlers do not need to be ob the object itself:

In [7]:
@ob.connect('foo')
def on_foo(*events):
    print('foo in other handler')

def on_bar(*events):
    print('bar in other handler')

ob.connect(on_bar, 'bar')  # "classic" connect method

received the foo event
received the foo event
received the bar event


<Handler 'on_bar' with 1 connections at 0x1e09a23ec88>

In [8]:
ob.emit('foo', {}); ob.emit('bar', {})

Dict([('type', 'bar'), ('source', <__main__.MyObject at 0x1e09a23e5c0>)])

### Properties
There are three constructs that automatically generate events. The first are properties.

In [9]:
class MyObject(event.HasEvents):

    @event.prop
    def foo(self, v=2):
        ''' This is a float indicating some value '''
        return float(v)
    
    @event.connect('foo')
    def on_foo(self, *events):
        print('foo changed to', events[-1].new_value)

ob = MyObject()

foo in other handler
received the foo event
received the bar event
bar in other handler


In [10]:
ob.foo = 7

foo changed to 2.0


In [11]:
print(ob.foo)

foo changed to 7.0
7.0


The function that is decorator is the setter function that is used to validate and normalize the input. The default value for `foo` above is `2`. An initial value can be provided like so:

In [12]:
ob = MyObject(foo=12)

### Readonly
Readonly properties are similar to normal properties, except that they cannot be set.

In [13]:
import time
class MyObject(event.HasEvents):

    @event.readonly
    def bar(self, v):  # no initial value
        ''' This is an int indicating some value '''
        return int(v)
    
    @event.connect('bar')
    def on_bar(self, *events):
        print('bar changed to', events[-1].new_value)
    
    def do_it(self):
        self._set_prop('bar', time.time())
    
ob = MyObject()

foo changed to 12.0


In [14]:
ob.do_it()

In [15]:
ob.do_it()

bar changed to 1465308112


### Emitters
Emitters make it easy to generate events from specific input (e.g. an event from another kind of event system) and act as a placeholder for the docs of public events.

In [16]:
class MyObject(event.HasEvents):

    @event.emitter
    def mouse_down(self, js_event):
        ''' Event emitted when the mouse is pressed down. '''
        return dict(button=js_event['button'])
    
    @event.connect('mouse_down')
    def on_bar(self, *events):
        for ev in events:
            print('detected mouse_down, button', ev.button)

ob = MyObject()

In [17]:
ob.mouse_down({'button': 1})
ob.mouse_down({'button': 2})

### Labels
Labels are a feature that makes it possible to infuence the order by
which event handlers are called, and provide a means to disconnect
specific (groups of) handlers. The label is part of the connection
string: 'foo.bar:label'.

In [18]:
class MyObject(event.HasEvents):

    @event.connect('foo:bb')
    def foo_handler1(*events):
        print('foo B')

    @event.connect('foo:cc')
    def foo_handler2(*events):
        print('foo C')
    
    @event.connect('foo:aa')
    def foo_handler3(*events):
        print('foo A')

ob = MyObject()

detected mouse_down, button 1
detected mouse_down, button 2


In [19]:
ob.emit('foo', {})

Dict([('type', 'foo'), ('source', <__main__.MyObject at 0x1e09a24f0f0>)])

In [20]:
ob.disconnect('foo:bb')
ob.emit('foo', {})

foo A
foo B
foo C


Dict([('type', 'foo'), ('source', <__main__.MyObject at 0x1e09a24f0f0>)])

### Dynamism

Dynamism is a concept that allows one to connect to events for which the source can change. It essentially allows events to be connected automatically, which greatly reduced boilerplate code. I makes it easy to connect different parts of an application in a robust way.

In [23]:
class Root(event.HasEvents):

    @event.prop
    def children(self, children):
        assert all([isinstance(child, Sub) for child in children])
        return tuple(children)
    
    @event.connect('children', 'children.*.count')
    def update_total_count(self, *events):
        total_count = sum([child.count for child in self.children])
        print('total count is', total_count)

class Sub(event.HasEvents):
    
    @event.prop
    def count(self, count=0):
        return int(count)

root = Root()
sub1, sub2, sub3 = Sub(count=1), Sub(count=2), Sub(count=3)
root.children = sub1, sub2, sub3

total count is 6


Updating the `count` property on any of its children will invoke the callback:

In [24]:
sub1.count = 100

total count is 105


We also connected to the `children` property, so that the handler is also invoked when the children are added/removed:

In [25]:
root.children = sub2, sub3

total count is 5


Naturally, when the count on new children changes ...

In [26]:
sub4 = Sub()
root.children = sub3, sub4

total count is 3


In [27]:
sub4.count = 10

total count is 13


In [28]:
sub1.count = 1000  # no update, sub1 is not part of root's children